In [4]:
!pip install -U tensorflow==2.* tf_slim
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 98785, done.
remote: Counting objects: 100% (1292/1292), done.
remote: Compressing objects: 100% (660/660), done.
remote: Total 98785 (delta 707), reused 1085 (delta 604), pack-reused 97493 (from 1)
Receiving objects: 100% (98785/98785), 622.31 MiB | 1022.00 KiB/s, done.
Resolving deltas: 100% (71700/71700), done.


In [3]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

[Errno 2] No such file or directory: 'models/research'
/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697357 sha256=17dc3e8a4dc63b0074f71bc272aa2aeaa42abb2fba8a44781f1e49bf1ead213c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ot1jt7nt/wheels/53/dd/70/2de274d6c443c69d367bd6a5606f95e5a6df61aacf1435ec0d
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1


In [ ]:
from pycocotools.coco import COCO
import numpy as np

annotation_file = ''
coco = COCO(annotation_file)